In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import pyarrow as pa

In [2]:
filename = '/data/cewalden/soc16S.parquet'
dfStr = pq.read_table(filename).to_pandas()
dfStr.head()

,ACS,AGER,ASSOC,BASE,CAT,CON,DECK,DET,DIV,FINC,...,FCONPR_F,LOTV_F,SQFS_F,FSQFS_F,PVALU_F,AUTH,COMP_D,SALE_D,STRT_D,AUTH_D
0,2,2,1,1,1,2,1,2,1,0,...,0,0,0,0,0,201510,2016-04-01,2015-09-01,2015-10-01,2015-10-01
1,1,2,1,1,1,1,1,2,1,1,...,0,0,0,0,0,201605,2016-10-01,2016-08-01,2016-05-01,2016-05-01
2,1,2,2,1,1,2,1,1,1,0,...,0,0,0,0,0,201506,2015-12-01,2016-01-01,2015-06-01,2015-06-01
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,201609,2017-03-01,2016-07-01,2016-09-01,2016-09-01
4,1,2,1,1,1,1,1,1,1,1,...,0,0,0,0,0,201511,2016-04-01,2016-04-01,2015-11-01,2015-11-01


In [3]:
qCols = ['AREA', 'FNSQ', 'FFNSQ', 'LOTV', 'PVALU', 'FSQFS', 'CONPR', 'SLPR', 'SQFS', 'WEIGHT', 'FCONPR', 'FSLPR']
    
dateCols = ['COMP_D', 'SALE_D', 'STRT_D', 'AUTH_D']
    
dCols = ['COMP', 'SALE', 'STRT', 'AUTH']

cCols = ['ACS', 'AGER', 'ASSOC', 'BASE', 'CAT', 'CON', 'DECK', 'DET', 'DIV',
       'FINC', 'FNBS', 'FOYER', 'FRAME', 'GAR', 'HEAT', 'HEAT2', 'LNDR',
       'METRO', 'MFGS', 'PATI', 'PRCH', 'SEWER', 'STOR', 'WAL1', 'WAL2',
       'WALS', 'WATER', 'BEDR', 'FPLS', 'FULB', 'HAFB', 'FUEL', 'FUEL2']

flags = ['AREA_F', 'FNSQ_F', 'SLPR_F', 'FSLPR_F', 
         'CONPR_F', 'FCONPR_F', 'LOTV_F', 
         'SQFS_F', 'FSQFS_F', 'PVALU_F', 'FFNSQ_F']

In [4]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

cat = 'CON'
y = dfStr[cat].values
X = dfStr.drop(dateCols, axis = 1).drop(cat, axis = 1).drop('WEIGHT', axis = 1).values

reg = LinearSVC(dual=False, tol = 1e-4)
cv_scores = cross_val_score(reg, X, y, cv=5)

print(cv_scores)

print("Average 5-Fold CV Score: {}".format(np.mean(cv_scores)))


[0.97838076 0.97947926 0.97925861 0.97992056 0.98102383]
Average 5-Fold CV Score: 0.9796126045559006


In [5]:
cat = 'DIV'
y = dfStr[cat].values
X = dfStr.drop(dateCols, axis = 1).drop(dCols, axis = 1).drop(cat, axis = 1).drop('WEIGHT', axis=1).values

reg = LinearSVC(dual=False, tol = 1e-4)
cv_scores = cross_val_score(reg, X, y, cv=5)

print(cv_scores)

print("Average 5-Fold CV Score: {}".format(np.mean(cv_scores)))

[0.28631254 0.28628143 0.29927168 0.29139073 0.25502318]
Average 5-Fold CV Score: 0.2836559133772131


In [7]:
def predictScores(X, y, regS, reps):
    scores = []
    for i in range(reps):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2)
        
        reg = regS
        reg.fit(X_train, y_train)
        reg.predict(X_test)
        scores.append(reg.score(X, y))
        
    print(scores)
    print("Average "+str(reps)+"-fold division score: " + str(np.array(scores).mean()))

In [5]:
cat = 'SLPR'
y = dfStr[dfStr[cat] > 0][cat].values
X = dfStr[dfStr[cat] > 0].drop(dateCols, axis = 1).drop(dCols, axis = 1).drop(cat, axis = 1).drop('WEIGHT', axis=1).values

regS = LinearSVC(dual=False, tol = 1e-4)
reps = 10
scores = []
#for i in range(reps):
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2)
    
reg = LinearSVC(dual=False, tol = 1e-4)
reg.fit(X_train, y_train)
reg.predict(X_test)
scores.append(reg.score(X, y))
        
print(scores)
print("Average "+str(reps)+"-fold division score: " + str(np.array(scores).mean()))

[0.05248710697055398]
Average 10-fold division score: 0.05248710697055398


In [12]:
#catDf = dfStr[dfStr['COMP'] > 0]
#catDf = catDf[catDf['STRT'] > 0]

#catDf['COMPTIME']= catDf['COMP'] - catDf['STRT']
cat = 'SLPR'
y = dfStr[cat].values
X = dfStr.drop(dateCols, axis = 1).drop(cat, axis=1).drop('WEIGHT', axis=1).values

predictScores(X,y,Lasso(alpha = 0.4, normalize=True),5)

[0.7244428285521067, 0.7244419336102512, 0.724462036488027, 0.7244021536118799, 0.7245239099775493]
Average 5-fold division score: 0.7244545724479627


Sales price has a good prediction 

In [9]:
cat = 'CAT'
y = dfStr[cat].values
X = dfStr.drop(dateCols, axis = 1).drop(dCols, axis = 1).drop(cat, axis=1).drop('WEIGHT', axis=1).values

predictScores(X,y,Lasso(alpha = 0.4, normalize=True),10)

[-3.535372605756315e-05, -1.0465139996806982e-06, -1.0463005011063231e-05, -1.837709753704786e-06, -3.150881207436562e-05, -7.21533206715641e-06, -7.051447952211021e-07, -1.613349725482216e-05, -1.8481996362895714e-05, -2.418643492374528e-06]
Average 10-fold division score: -1.251643808688474e-05
